In [1]:
import os
os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

from heisembergs import sparse_xxz_hamiltonian
import sunny.shannon as ashannon
import scipy.sparse as sparse
import numpy as np

In [2]:
def xx_num_an_comp_debrief(N_range, antiferro, spin_penalty_factor, comparison_tol, ndecimals=5, print_mode=False):
    # Numerical
    N_arr = np.arange(*N_range, 2)
    Egs_num_arr = np.zeros_like(N_arr, dtype=float)
    max_prob_num_arr = np.zeros_like(N_arr, dtype=float)
    n_fermions_num_arr = np.zeros_like(N_arr)
    n_max_conf_num_arr = np.zeros_like(N_arr)
    for i, N in enumerate(N_arr):
        if print_mode: print(f"\rNum: N = {N}", end="")
        H = sparse_xxz_hamiltonian(0, antiferro, N, spin_penalty_factor)
        evals, evecs = sparse.linalg.eigsh(H, which="SA", k=1, tol=1e-8)
        Egs_num_arr[i] = evals[0]
        amps = np.abs(evecs[:, 0])**2
        max_amp_ind = np.argmax(amps)
        max_prob_num_arr[i] = amps[max_amp_ind]
        n_fermions_num_arr[i] = np.sum(np.array(list(f"{max_amp_ind:0{N}b}"), dtype=int))
        n_max_conf_num_arr[i] = np.sum(np.isclose(amps, amps[max_amp_ind], atol=comparison_tol))

    # Analytical
    prev_eps_ashannon = ashannon.epsilon
    ashannon.epsilon = comparison_tol
    Egs_an_arr = np.zeros_like(N_arr, dtype=float)
    max_prob_an_arr = np.zeros_like(N_arr, dtype=float)
    n_fermions_an_arr = np.zeros_like(N_arr)
    n_max_conf_an_arr = np.zeros_like(N_arr)
    for i, N in enumerate(N_arr):
        if print_mode: print(f"\rAn:  N = {N}", end="")
        Egs_an_arr[i] = ashannon.ground_state_energy(N)
        max_probs_obj = ashannon.max_probs(N)
        max_prob_an_arr[i] = max_probs_obj[0].prob
        n_fermions_an_arr[i] = len(max_probs_obj[0].conf)
        n_max_conf_an_arr[i] = len(max_probs_obj)
    ashannon.epsilon = prev_eps_ashannon

    # Print results
    print("\r-------------------------------------------------------------------------------------------------------")
    for i, N in enumerate(N_arr):
        N_str = f"N = {N}"
        first_line = "\t".join([N_str, "Num:", f"Egs = {Egs_num_arr[i]:.0{ndecimals}f}", f"max p = {max_prob_num_arr[i]:.0{ndecimals}f}", f"n_ferm = {n_fermions_num_arr[i]:.0{ndecimals}f}", f"n_conf = {n_max_conf_num_arr[i]:.0{ndecimals}f}"])
        second_line = "\t".join([" "*len(N_str), "An: ", f"Egs = {Egs_an_arr[i]:.0{ndecimals}f}", f"max p = {max_prob_an_arr[i]:.0{ndecimals}f}", f"n_ferm = {n_fermions_an_arr[i]:.0{ndecimals}f}", f"n_conf = {n_max_conf_an_arr[i]:.0{ndecimals}f}"])
        print(first_line)
        print(second_line)
        print("\r-------------------------------------------------------------------------------------------------------")

In [3]:
N_range = [7, 19]
delta = 0
antiferro = False
spin_penalty_factor = 10 # Default = 10
comparison_tol = 1e-6
ndecimals = 5

xx_num_an_comp_debrief(N_range, antiferro, spin_penalty_factor, comparison_tol, ndecimals, print_mode=True)

-------------------------------------------------------------------------------------------------------
N = 7	Num:	Egs = -8.98792	max p = 0.06626	n_ferm = 4.00000	n_conf = 7.00000
     	An: 	Egs = -4.49396	max p = 0.06626	n_ferm = 3.00000	n_conf = 7.00000
-------------------------------------------------------------------------------------------------------
N = 9	Num:	Egs = -11.51754	max p = 0.03106	n_ferm = 5.00000	n_conf = 9.00000
     	An: 	Egs = -5.75877	max p = 0.03106	n_ferm = 5.00000	n_conf = 9.00000
-------------------------------------------------------------------------------------------------------
N = 11	Num:	Egs = -14.05335	max p = 0.01476	n_ferm = 6.00000	n_conf = 11.00000
      	An: 	Egs = -7.02667	max p = 0.01476	n_ferm = 5.00000	n_conf = 11.00000
-------------------------------------------------------------------------------------------------------
N = 13	Num:	Egs = -16.59246	max p = 0.00707	n_ferm = 7.00000	n_conf = 13.00000
      	An: 	Egs = -8.29623	max p = 0.00707	

In [4]:
# Check of the number of fermions
N_range = [1, 19]
print(list(range(1, 19, 2)))
print([2*(N//4 + 1) - 1 for N in range(1, 19, 2)])

[1, 3, 5, 7, 9, 11, 13, 15, 17]
[1, 1, 3, 3, 5, 5, 7, 7, 9]
